In [2]:
from googletrans import Translator
from concurrent.futures import ThreadPoolExecutor
import re
import time

In [3]:
def backtranslate(text, src_lang='ko', target_lang='en', max_retries=3):
    translator = Translator()
    text_lines = text.split('\n')
    result_lines = []

    for line in text_lines:
        person = line[:11]
        dia = line[11:]
        
        for attempt in range(max_retries):
            try:
                translate_text = translator.translate(dia, src=src_lang, dest=target_lang).text
                back_translate_text = translator.translate(translate_text, src=target_lang, dest=src_lang).text
                result_lines.append(person + back_translate_text)
                break
            except Exception as e:
                if attempt == max_retries - 1:
                    print(f"Translation failed after {max_retries} attempts: {e}")
                    return None
                time.sleep(1)  # API 호출 사이 대기

    return '\n'.join(result_lines)

In [4]:
def process_batch(batch):
    augmented_rows = []
    for index, row in batch.iterrows():
        dia = backtranslate(text=row['dialogue'])

        if dia is not None:
            new_row = row.copy()
            new_row['fname'] = f'train_{len(batch) + len(augmented_rows)}'
            new_row['dialogue'] = dia
            augmented_rows.append(new_row)

    return augmented_rows

def batch_backtranslate(df, batch_size=100, num_threads=4):
    augmented_rows = []
    total_batches = len(df) // batch_size + (len(df) % batch_size != 0)  # Total number of batches
    start_time = time.time()

    # Create a ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = []

        # Process each batch in parallel
        for batch_idx in range(0, len(df), batch_size):
            batch = df.iloc[batch_idx:batch_idx + batch_size]
            futures.append(executor.submit(process_batch, batch))

        # Collect results as they are completed
        for future in futures:
            augmented_rows.extend(future.result())

    total_time = time.time() - start_time
    print(f"Total processing time: {total_time / 60:.2f} minutes")

    return pd.DataFrame(augmented_rows)

In [7]:
df = pd.read_csv("/root/NLP/data/train.csv")

In [8]:
input_text = df.head()

backtranslated_text = batch_backtranslate(input_text)
backtranslated_text

Total processing time: 0.36 minutes


,fname,dialogue,summary,topic
0,train_5,"#Person1#: 안녕하세요, 스미스 씨. 저는 호킨스 의사입니다. 오늘 왜 왔니...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_6,"#Person1#: 안녕하세요, 파커 부인, 잘 지내세요?\n#Person2#: 안...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
2,train_7,#Person1#: 실례합니다. 키 번들을 본 적이 있습니까?\n#Person2#:...,"#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,train_8,#Person1#: 왜 여자 친구가 있다고 말하지 않았습니까?\n#Person2#:...,#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다
4,train_9,"#Person1#: 안녕하세요, 숙녀 여러분! 오늘 밤 정말 멋져요. 나와 함께이 ...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스


In [7]:
df

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
2,train_2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,train_3,#Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...,#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스
...,...,...,...,...
91749,train_91749,"#Person1#: 엄마, 다음 토요일에 이 삼촌네 가족을 방문하기 위해 비행기를 ...",#Person1#은 다음 토요일에 이 삼촌네를 방문할 때 가방을 어떻게 싸야 할지 ...,짐 싸기
91750,train_91750,"#Person1#: 엄마, 다음 가방을 이 삼촌네 가족을 방문하기 위해 비행기를 타...",#Person1#은 다음 토요일에 이 삼촌네를 방문할 때 가방을 어떻게 싸야 할지 ...,짐 싸기
91751,train_91751,"#Person1#: 엄마, 다음 토요일에 이 삼촌네 가족을 방문하기 위해 비행기를 ...",#Person1#은 다음 토요일에 이 삼촌네를 방문할 때 가방을 어떻게 싸야 할지 ...,짐 싸기
91752,train_91752,"#Person1#: 엄마, 갈 토요일에 이 삼촌네 가족을 방문하기 위해 비행기를 타...",#Person1#은 다음 토요일에 이 삼촌네를 방문할 때 가방을 어떻게 싸야 할지 ...,짐 싸기


In [8]:
# 데이터 증강
augmented_df = batch_backtranslate(df)
augmented_df.to_csv('nltk_bt_data.csv', index=False)

KeyboardInterrupt: 

In [ ]:
df.to_csv("/root/NLP/data/aug_nltk_2_cleaned_bt_train.csv", index=False)